In [1]:
!ls /content/

drive  sample_data


In [2]:
import re
import random

# --- 1. tarot_knowledge.txt の読み込みと解析 (前回修正したものをそのまま使用) ---
def load_and_parse_tarot_data(file_path='/content/drive/MyDrive/生成AIとビジネス応用/MyProject/tarot_knowledge.txt'):
    """
    tarot_knowledge.txtを読み込み、カードごとの辞書のリストに変換する関数
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    card_blocks = content.strip().split('card:')[1:]

    tarot_database = []
    for block in card_blocks:
        lines = [line.strip() for line in block.strip().split('\n') if line.strip()]
        if not lines: continue

        card_info = {'name': lines[0]}
        for line in lines[1:]:
            if line.startswith('keywords:'):
                card_info['keywords'] = line.replace('keywords:', '').strip()
            elif line.startswith('symbol:'):
                card_info['symbol'] = line.replace('symbol:', '').strip()

        if 'name' in card_info and 'keywords' in card_info and 'symbol' in card_info:
            tarot_database.append(card_info)

    return tarot_database

# --- 実行デモ ---
try:
    # 1. カードデータベースをロード
    tarot_db = load_and_parse_tarot_data()
    print(f"✅ {len(tarot_db)}枚のカード情報を正常に読み込みました。")

    # 2. 占い師のインスピレーション（印象）を設定
    # ここのテキストを「犬が好きそう」「猫っぽい人」などに変えてみてください
    inspiration_input = "どことなく猫っぽい雰囲気の人"

    # 3. カードをランダムに1枚引く (0から21の乱数を生成)
    drawn_card_index = random.randint(0, len(tarot_db) - 1)
    drawn_card_info = tarot_db[drawn_card_index]

    print("\n--- 占い結果 ---")
    print(f"🔮 占い師の印象: 「{inspiration_input}」")
    print(f"🃏 引いたカード: {drawn_card_info['name']}")
    print("--------------------")

    # 4. ファインチューニング済みLLMに渡すプロンプトを作成
    # このプロンプトをLLMに入力すると、最終的な画像生成プロンプトが出力される
    final_prompt_for_llm = f"""### Instruction:
以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

### Drawn Card Info:
- Name: {drawn_card_info['name']}
- Keywords: {drawn_card_info['keywords']}
- Symbols: {drawn_card_info['symbol']}

### Fortune Teller's Impression:
{inspiration_input}

### Output:
"""

    print("\n--- LLMへの最終入力プロンプト ---")
    print(final_prompt_for_llm)
    print("---------------------------------")


except FileNotFoundError:
    print("❌ エラー: tarot_knowledge.txt が見つかりません。Colabにアップロードしたか確認してください。")
except Exception as e:
    print(f"❌ 予期せぬエラーが発生しました: {e}")

✅ 22枚のカード情報を正常に読み込みました。

--- 占い結果 ---
🔮 占い師の印象: 「どことなく猫っぽい雰囲気の人」
🃏 引いたカード: 5 - 教皇 (The Hierophant)
--------------------

--- LLMへの最終入力プロンプト ---
### Instruction:
以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

### Drawn Card Info:
- Name: 5 - 教皇 (The Hierophant)
- Keywords: 伝統, 慈悲, 権威, 信仰, 教え, 導き, 社会性
- Symbols: 3層の冠, 3段の十字架の杖, 祝福する右手, 2人の修道士, 交差した鍵

### Fortune Teller's Impression:
どことなく猫っぽい雰囲気の人

### Output:

---------------------------------


In [8]:
# --- 1. 必要なライブラリのインストール ---
!pip install -q transformers "datasets[progress]" "accelerate>=0.21.0" "peft>=0.4.0" "bitsandbytes>=0.40.2" "trl>=0.4.7"

print("✅ ライブラリのインストールが完了しました。")

# --- 2. データの準備 ---
import pandas as pd
from datasets import Dataset, DatasetDict
import json # jsonを読み込むために追加

# dataset.jsonlをpandasで読み込み
try:
    df = pd.read_json('/content/drive/MyDrive/生成AIとビジネス応用/MyProject/dataset.jsonl', lines=True)

    # データを整形する関数を新しい仕様に書き換える
    def format_prompt(example):
        # JSON形式のinputを辞書に変換
        input_data = json.loads(example['input'])
        card = input_data['card']
        impression = input_data['impression']

        # 最終的な推論時と全く同じ形式のプロンプトを作成
        return f"""### Instruction:
    以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

    ### Drawn Card Info:
    - Name: {card}

    ### Fortune Teller's Impression:
    {impression}

    ### Output:
    {example['output']}"""

    # データを整形する関数を新しい仕様に書き換える
    def format_prompt(example):
        # 新しいデータセットの 'input' を使うように変更
        return f"""### Instruction:
      以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

      ### Input:
      {example['input']}

      ### Output:
      {example['output']}"""

    # データセット全体に適用
    df['text'] = df.apply(format_prompt, axis=1)

    # Dataset形式に変換
    dataset = Dataset.from_pandas(df[['text']])

    print("\n✅ dataset.jsonl を正常に読み込み、整形しました。")
    print("データサンプル (整形後):")
    print(dataset[0]['text'])

except FileNotFoundError:
    print("❌ エラー: dataset.jsonl が見つかりません。Colabにアップロードしたか確認してください。")


# --- 3. ファインチューニングの実行 ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# 4bit量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# モデルとトークナイザーのロード
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0} # GPUにロード
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# パディングトークンがないモデルのための設定
tokenizer.pad_token = tokenizer.eos_token

# LoRAの設定
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    # TinyLlamaのターゲットモジュールを指定
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# トレーニング引数の設定
training_arguments = TrainingArguments(
    output_dir="./results",
    report_to="none",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_steps=50,
    logging_steps=10,
    num_train_epochs=30, # エポック数を指定 (データが少ないので3周させる)
    max_steps=-1,       # max_stepsの代わりにエポック数で制御
    fp16=True,
)

# SFTTrainerの設定
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

print("\n⏳ ファインチューニングを開始します...(10~15分程度かかります)")
# 学習の実行
trainer.train()
print("✅ ファインチューニングが完了しました！")

# --- 4. 学習済みモデルで推論を試す ---
from transformers import pipeline

# 占い師の印象と引いたカード情報を準備
inspiration_input = "どことなく猫っぽい雰囲気の人"
drawn_card_name = "8 - 力 (Strength)"

# モデルへの最終的な指示プロンプトを作成
prompt_for_model = f"""### Instruction:
以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

### Drawn Card Info:
- Name: {drawn_card_name}

### Fortune Teller's Impression:
{inspiration_input}

### Output:
"""

print("\n--- 推論テスト ---")
print("モデルへの入力:\n" + prompt_for_model)
print("\nモデルの出力:")

# 推論パイプラインを作成
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt_for_model)
print(result[0]['generated_text'])

✅ ライブラリのインストールが完了しました。

✅ dataset.jsonl を正常に読み込み、整形しました。
データサンプル (整形後):
### Instruction:
      以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

      ### Input:
      {"card": "9 - 隠者 (The Hermit)", "impression": "甘いものが好きな人"}

      ### Output:
      a single tarot card, 'The Hermit', a wise old man on a mountain peak, his lantern glows warmly, instead of a star, it contains a delicious floating cupcake, his expression is one of quiet joy, surrounded by a magical candy landscape


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20 | Num Epochs = 30 | Total steps = 90
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 50,462,720 of 1,150,511,104 (4.39% trained)



⏳ ファインチューニングを開始します...(10~15分程度かかります)


Step,Training Loss
10,1.898500
20,0.925200
30,0.647600
40,0.392000
50,0.177500
60,0.085500
70,0.055200
80,0.046800
90,0.044500


Device set to use cuda:0


✅ ファインチューニングが完了しました！

--- 推論テスト ---
モデルへの入力:
### Instruction:
以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

### Drawn Card Info:
- Name: 8 - 力 (Strength)

### Fortune Teller's Impression:
どことなく猫っぽい雰囲気の人

### Output:


モデルの出力:
### Instruction:
以下の「引いたカードの情報」と「占い師の印象」を創造的に組み合わせて、最高の画像生成プロンプトを作成してください。

### Drawn Card Info:
- Name: 8 - 力 (Strength)

### Fortune Teller's Impression:
どことなく猫っぽい雰囲気の人

### Output:
一條たたかったカード、「力」の人が、アーキテクチャや構造のようなものを試みるのが好きな人」を創造的に組み合わ
